In [ ]:
!pip install sentence-transformers transformers faiss-cpu
!pip install accelerate  # ถ้าใช้ GPU
!pip install cross-encoder
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement cross-encoder (from versions: none)
ERROR: No matching distribution found for cross-encoder
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
import json

# Load JSON knowledge base
with open("/content/drive/MyDrive/Medical_Appication/New_data_web/ADA_data_.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

print(f"Loaded {len(documents)} documents from knowledge base")

Loaded 4560 documents from knowledge base


In [ ]:
# ดูตัวอย่าง document
print(documents[0])

{'title': 'Diabetes Research, Education, Advocacy | ADA', 'content': "Learn your risk for type 2 and take steps to prevent it Learn the ins and outs of diabetes and steps for better care Obesity care is as unique as you are. Discover the different treatment and prevention options. Learn what you can do to keep your child with diabetes safe at school What you need to know about gestational diabetes Make a plan to be prepared when illness strikes Sign up to receive timely information on topics to help you and your family live a healthy life Bringing ADA's evidence-based guidelines and perspective directly to you in the store so you can make informed choices. Take the 60-second test to find out your risk Find help and resources in your zip code Choose between a one-time donation or recurring monthly gifts. Reach out and get involved in programs and events near you. We make vehicle donations easy so you can help fuel our fight, hassle-free. Time is running out to double your impact! Honor 

In [ ]:
import re
import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
def chunk_text(text, max_words=150, overlap=30):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + max_words
        chunks.append(" ".join(words[start:end]))
        start += max_words - overlap
    return chunks

texts = []
metadata = []

for doc in documents:  # documents = [{'title','source','content'}]
    content_list = doc.get("content") if "content" in doc else [doc.get("text","")]
    for content in content_list:
        if not content.strip():
            continue
        clean_content = re.sub(r"\s+", " ", content)
        chunks = chunk_text(clean_content)
        for chunk in chunks:
            texts.append(chunk)
            metadata.append({
                "title": doc.get("title",""),
                "source": doc.get("source",""),
                "text": chunk
            })

print("Total chunks:", len(texts))

Total chunks: 6796820


In [ ]:
embedding_model_name = "Qwen/Qwen3-Embedding-0.6B"
device = "cuda" if torch.cuda.is_available() else "cpu"

model_emb = SentenceTransformer(
    embedding_model_name,
    device=device,
    # torch_dtype=torch.float16 # Removed torch_dtype as it is not a valid argument
)

def get_embedding(texts):
    if isinstance(texts, str):
        texts = [texts]
    return model_emb.encode(texts, convert_to_numpy=True, normalize_embeddings=True).astype("float32")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
batch_size = 1024
all_embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    emb = get_embedding(batch)
    all_embeddings.append(emb)

all_embeddings = np.vstack(all_embeddings).astype("float32")
print("Embeddings shape:", all_embeddings.shape)


KeyboardInterrupt: 

In [ ]:
dimension = all_embeddings.shape[1]
nlist = 4096  # clusters
quantizer = faiss.IndexFlatIP(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

# GPU
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

# Train + Add
gpu_index.train(all_embeddings)
gpu_index.add(all_embeddings)
gpu_index.nprobe = 20  # เพิ่ม nprobe เพื่อ accuracy balance speed
print("FAISS index created with", len(texts), "vectors")

# Save/load
faiss.write_index(faiss.index_gpu_to_cpu(gpu_index), "faiss_index.ivf")

In [ ]:
reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank(question, retrieved_docs, top_n=5):
    pairs = [(question, doc["text"]) for doc in retrieved_docs]
    scores = reranker_model.predict(pairs)
    ranked_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)]
    return ranked_docs[:top_n]

In [ ]:
llm_model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer_llm = AutoTokenizer.from_pretrained(llm_model_name, trust_remote_code=True)
model_llm = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
)

qa_pipeline = pipeline(
    "text-generation",
    model=model_llm,
    tokenizer=tokenizer_llm,
    max_new_tokens=400
)

In [ ]:
def retrieve_documents(question, top_k=10):
    q_emb = get_embedding(question).astype("float32")
    distances, indices = gpu_index.search(q_emb, top_k)
    results = []
    for idx in indices[0]:
        if idx < len(metadata):
            results.append(metadata[idx])
        else:
            results.append({"title":"","source":"","text":""})
    return results

def build_prompt_fewshot_cot(question, retrieved_docs):
    context_texts = ""
    for doc in retrieved_docs:
        context_texts += f"Title: {doc['title']}\nContent: {doc['text']}\nSource: {doc['source']}\n\n"

    few_shot_examples = """
Example 1:
Question: What foods are recommended for people with diabetes?
Answer:
Step 1: Identify the goal: Maintain stable blood sugar levels.
Step 2: Check nutrition sources that help: vegetables, whole grains, lean proteins.
Step 3: Avoid foods that spike blood sugar: sugary foods, sweet drinks.
Final Answer: People with diabetes should eat vegetables, whole grains, lean proteins, and limit sugary foods. Source: CDC Diabetes Guidelines

Example 2:
Question: อาหารหรือเครื่องดื่มอะไรที่ช่วยลดความเสี่ยงการเกิดเบาหวานชนิด 2 ได้?
Answer:
Step 1: เป้าหมาย: ลดความเสี่ยงการเกิดเบาหวานชนิด 2 และควบคุมน้ำหนัก
Step 2: แนะนำอาหาร/เครื่องดื่ม: ผัก ผลไม้ ธัญพืชไม่ขัดสี โปรตีนไม่ติดมัน น้ำเปล่า ชาเขียวไม่หวาน
Step 3: เพิ่มเติม: หลีกเลี่ยงอาหารน้ำตาลสูง ออกกำลังกายสม่ำเสมอ
Final Answer: รับประทานผัก ผลไม้ ธัญพืชไม่ขัดสี โปรตีนไม่ติดมัน และน้ำเปล่าหรือชาเขียวไม่หวาน ควบคุมอาหารและออกกำลังกายสม่ำเสมอ. Source: CDC, ADA
"""
    prompt = f"""
You are a bilingual medical assistant specialized in diabetes.
Answer the question using ONLY the provided context.
Provide step-by-step reasoning as shown in examples.
If the answer is not in the context, reply "Information not available".

{few_shot_examples}

Question: {question}

Context:
{context_texts}

Answer:
"""
    return prompt


In [ ]:
def answer_question_with_rerank(question, top_k=10, rerank_top=5):
    docs = retrieve_documents(question, top_k)
    docs = rerank(question, docs, top_n=rerank_top)
    prompt = build_prompt_fewshot_cot(question, docs)
    response = qa_pipeline(prompt, max_new_tokens=250)[0]['generated_text']
    return response

In [ ]:
questions = [
    "ผู้ป่วยโรคเบาหวานสามารถออกกำลังกายผระเภทใดได้บ้าง?",
    "อาหารอะไรบ้างที่ผู้ป่วยโรคเบาหวานควรหลีกเลี่ยง?",
    "ทำไมผู้ป่วยเบาหวานต้องฉีด อินซูลิน (Insulin)?",
    "โรคเบาหวานสามารถทำให้เกิดภาวะแทรกซ้อนทางระบบประสาทตาได้หรือไม่?",
    "ภาวะน้ำตาลในเลือดต่ำคืออะไรและต้องทำอย่างไร?",
    "What is the main difference between Type 1 and Type 2 diabetes?",
    "What foods are recommended for people with diabetes?",
    "What is the role of insulin in the body?",
    "What happens if a person's blood sugar is too high (hyperglycemia)?"
]

for q in questions:
    print("Question:", q)
    print("Answer:\n", answer_question_with_rerank(q))
    print("-"*80)

In [ ]:
!pip install qwen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.8/533.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [ ]:
+pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install sentence-transformers  # ถ้าจะใช้ embedding ด้วย
pip install chromadb  # ถ้าอยากเก็บ embeddings + vector DB
